### Tutorial 1: Prompt Engineering and Agents

The LLM is a function `(input: string) -> string` that can be used for any purpose. Prompt engineering lets us do better than strings, more like: `(input: PromptTemplate) -> AgentResponse`.

**Prompt Engineering:** pretty much any LLM prompt that gets output beyond the normal completion/chat behavior:

* General-purpose NLP tasks: categorization, clustering, entity extraction, summarization, translation... Pretty much everything [More in Example #1]. Or playing D&D, pretending to be a bash shell, etc.

* Or large improvements on problem-solving benchmarks by including simple phrases like "Lets think step by step" (Large Language Models are Zero-Shot Reasoners: https://arxiv.org/abs/2205.11916)

* Or Jailbreaks! (DAN prompt: https://github.com/0xk1h0/ChatGPT_DAN) - Not all fun and games, these could become a serious security concern as LLMs become more widely adopted as tools...

**Agent:** A piece of code that queries the LLM with a prompt engineered to give structured output. The code usually parses the output and then takes some action before returning a final response. The actions the code takes can be iterative (repeatedly prompting the LLM) until the final response is determined. *The LLM as a tool.*

* "By conditioning on natural language instructions, large language models (LLMs) have displayed impressive capabilities as general-purpose computers." (Large Language Models Are Human-Level Prompt Engineers; https://arxiv.org/abs/2211.01910)

* "Techniques to improve reliability" and its citations is the best starting point (https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md)

* Langchain, AutoGPT, JARVIS, even ChatGPT plugins are all built on this core idea.

*(NOTE: Follow the normal setup instructions in README.md before running this notebook!)*



In [1]:
# hack to make imports from src/ work (skip ahead)
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

**The Chat Completion API**

In [2]:
from llm import chat_completion # <-- gpt-toolbox's small wrapper around the openai ChatCompletion API

# the API gives us 3 parts to the prompt: system, examples, and user (https://platform.openai.com/docs/api-reference/chat)

system = ""
examples = []
user = "Hi?"

print(chat_completion(system, examples, user))

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1683231674,
  "id": "chatcmpl-7CZjuzxRmzXRWiQtVH9Y0QtQ5z6fn",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 15,
    "total_tokens": 24
  }
}


**Example #1: A prompt for NLP tasks**

Lets "engineer a prompt" to get GPT to categorize, summarize, extract entities, and translate to french.

In [3]:
system = """
Given the user's input, output:
Category:<one of the following: "news", "science", "politics", "sports", "entertainment", "business", "health", "technology", "world">
Summary:<one sentence summary>
Entities:<list of entities, comma separated>
French:<French translation>
"""

examples = []

def nlp_tasks_template(input): return f"""
Input: {input}
Category:
"""

q = "Ice cream trucks are a ubiquitous part of summer in New York, and a staple at city parks. So if you live right by a park, the trucks, with their incessant jingles and rattling generators, come with the territory. However, the truck operators, like other food vendors, must be licensed and permitted, and comply with city rules and regulations. "

response = chat_completion(system, examples, nlp_tasks_template(q))

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Category: Business\nSummary: Ice cream truck operators in New York must be licensed and permitted to comply with city rules and regulations.\nEntities: New York, ice cream trucks, park, food vendors, city rules and regulations\nFrench: Les op\u00e9rateurs de camions de cr\u00e8me glac\u00e9e \u00e0 New York doivent \u00eatre autoris\u00e9s et autoris\u00e9s \u00e0 se conformer aux r\u00e8gles et r\u00e8glements de la ville.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683231675,
  "id": "chatcmpl-7CZjv3MzpOjvO5USXis04nwdanuE0",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 87,
    "prompt_tokens": 157,
    "total_tokens": 244
  }
}


Since it's structured output, we can parse it with a regex

In [14]:
import re
import json

def parse_nlp_tasks_completion(response):
    completion = response.choices[0].message.content
    output_pattern = re.compile(
        r"Category:(?P<Category>[^\n]+)\n"
        r"Summary:(?P<Summary>[^\n]+)\n"
        r"Entities:(?P<Entities>[^\n]+)\n" 
        r"French:(?P<French>[^\n]+)\n?")

    match = output_pattern.match(completion)

    return match.groupdict()

parsed = parse_nlp_tasks_completion(response)

parsed = json.dumps(parsed, indent=4)

print(parsed)

{
    "Category": " Business",
    "Summary": " Ice cream truck operators in New York must be licensed and permitted to comply with city rules and regulations.",
    "Entities": " New York, ice cream trucks, park, food vendors, city rules and regulations",
    "French": " Les op\u00e9rateurs de camions de cr\u00e8me glac\u00e9e \u00e0 New York doivent \u00eatre autoris\u00e9s et autoris\u00e9s \u00e0 se conformer aux r\u00e8gles et r\u00e8glements de la ville."
}


And build on top of it arbitrarily...

In [9]:
def to_french(text):
    response = chat_completion(system, examples, nlp_tasks_template(text))
    parsed = parse_nlp_tasks_completion(response)
    return parsed["French"]

print(to_french("I like ice cream."))

 J'aime la crème glacée.


With about 10 lines of code, we have a function that translates anything to French. Or, extract entities:

In [12]:
def extracy_entities(text):
    response = chat_completion(system, examples, nlp_tasks_template(text))
    parsed = parse_nlp_tasks_completion(response)
    return parsed["Entities"]

print(extracy_entities("I like to eat ice cream in Paris and Pizza in New York."))

 ice cream, Paris, Pizza, New York


### Summary of Core Ideas

1. The LLM is a function `(input: string) -> string`
2. "Prompt Engineering" is the act of getting special output from the LLM. One application of this is getting structured (ie, parseable) output.
3. We can easily build programs that hit the LLM with prompts designed for specialized purposes, parse the output, and do stuff.
4. *The LLM is a tool*